In [1]:
# Leer todas las funciones del algoritmo QR del examen pasado
# La función creada para este proyecto se llama eigenvalores_QR, se encuentra en la siguiente sección

In [2]:
import numpy as np
import copy
import codecs
import sys
from scipy.linalg import solve_triangular

def busca_ayuda(cadena):
    """
    Función que devuelve el texto de ayuda correspondiente a la función
    que se pase como parámetro, obtenida de un unico archivo .txt
    donde están documentadas las "ayudas" de todas las funciones asociadas
    a la factorización QR (contenidas en este .py)
    
    params: cadena  nombre de la función de la que se desea obtener ayuda
                    
    return: help    ayuda de la función buscada en forma de texto
    
    """
    l=len(cadena)
    help=codecs.open('Help_funciones_factorizacion_QR.txt', "r", "utf-8").read()
    p_inicio=help.find("****i****" + cadena)
    p_final=help.find("****f****" + cadena)
    help=help[(p_inicio+9+l):p_final]
    return help






def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstance(minimo_valor, (int, float))==False:
        sys.exit('Los parámetros maximo_valor y minimo_valor deben ser numericos')

    #Se inicializa una matriz llena de ceros con las dimensiones deseadas (mxn)
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            #Si entero es verdadero se obtiene el maximo entero menor o igual a 1 (//1)
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

#crear_matriz_aleatoria.__doc__ =busca_ayuda("crear_matriz_aleatoria")






def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

#house.__doc__ =busca_ayuda("house")






def matriz_auxiliar_Arv(A,ind_singular=False):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columnas
    m,n=A.shape
    #se crea una matriz con los valores de A
    Arv=copy.copy(A)
    
    #si no se pide que se cheque que la matriz es singular, se obtendrá la matriz R
    #aún si esta es singula (que exista algún elemento de la diagonal que sea 0
    if ind_singular==False:
        for j in range(n):
            beta, v=house(Arv[j:m,j])
            #Con esta multiplicación se van generando las componentes r de la matriz R
            Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
            #se guarda en cada columnas los valores de v d, excepto la primer componente (que vale 1)
            Arv[(j+1):m,j]=v[1:(m-j)]
    else:
        for j in range(n):
            beta, v=house(Arv[j:m,j])
            Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
            #Si se detecta que alguna entrada de la diagonal (elemento de la maatriz R)
            #es cero, se devuelve un valor nulo (None), y para el cálculo de Arv
            #No se condiciona sobre si alcanza el valor de cero, sino más bien
            #si está lo suficientemente cercano, pues dados los errores por redondeo
            #no siempre se alcanza el cero aunque teóricamente sí lo sea, pero el
            #algoritmos arroja un valor cercano a cero
            if Arv[j,j]<10**(-14) and Arv[j,j]>-10**(-14):
                return None
            Arv[(j+1):m,j]=v[1:(m-j)]
    return Arv

#matriz_auxiliar_Arv.__doc__ =busca_ayuda("matriz_auxiliar_Arv")






def matriz_Q_R(A,ind_singular=False):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)
    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    #si ind_singular es True, entonces no se devuelve la matriz R ni Q
    #esto se plantea así como un sistema de contról, para evitar comprobar que las matrices
    #sean singulares (y no tengan una única solución) mediante el determinante, que
    #involucra un alto costo computacional para matrices de tamaño representativo
    Arv=matriz_auxiliar_Arv(A,ind_singular)

    if Arv is None:
        return None, None
    else:
        m,n=A.shape
        Q=np.eye(m)
        R=copy.copy(A)
        for j in range(n):
            Qj=Q_j(Arv,j+1)
            Q=Q@Qj
            R=Q_j(Arv,j+1)@R
        return Q,R

#matriz_Q_R.__doc__ =busca_ayuda("matriz_Q_R")






def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j
    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para construir Q_j requerimos usar el vector v contenido en Arv contenido
    #en la j-1 columna (considerando que en Python la primer columna es la cero)
    v=np.concatenate((1,Arv[j:m,(j-1)]), axis=None)
    beta=2/(1+Arv[j:m,(j-1)].dot(Arv[j:m,(j-1)]))
    Qj[(j-1):m,(j-1):m]=np.eye(m-(j-1))-beta*np.outer(v,v)
    return Qj

#Q_j.__doc__ =busca_ayuda("Q_j")






def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")

    #Si la matriz A es singular, Q y R serán igual a None
    Q,R=matriz_Q_R(A,1)
    if Q is None:
        sys.exit('La matriz A asociada al sistema de ecuaciones es singular')
    b_prima=np.transpose(Q)@b
    x=solve_triangular(R, b_prima,lower=False)
    return x

#Solucion_SEL_QR_nxn.__doc__ =busca_ayuda("Solucion_SEL_QR_nxn")





def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")    
    
    if m1==False:
        m1=m//2
    if n1==False:
        n1=n//2
    
    b1 = b[:m1]
    b2 = b[m1:m]
    A11 = A[:m1,:n1]
    A21 = A[m1:m,:n1]
    A12 = A[:m1,n1:n]
    A22 = A[m1:m,n1:n]

    return A11,A21,A12,A22,b1,b2

#crear_bloques.__doc__ =busca_ayuda("crear_bloques")






def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")
    
    if m1==False:
        m1=m//2
    if n1==False:
        n1=n//2

    if m != n:
        sys.exit('A debe ser cuadrada')

#     if np.linalg.det(A)==0:
#         sys.exit('A debe ser no singular')

    A11,A21,A12,A22,b1,b2=crear_bloques(A,b,m1,n1)

    # 1. Calcular Y=A11^{-1}A12 y y=A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    #if np.linalg.det(A11)==0:
    #    sys.exit('A11 debe ser no singular')
    y=Solucion_SEL_QR_nxn(A11,b1)
    
    Y=np.zeros((n1,n-n1))
    for j in range(n-n1):
        Y[:,j]=Solucion_SEL_QR_nxn(A11,A12[:,j])
    
    # 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S=A22-A21@Y
    b_h=b2-A21@y

    # 3. Resolver Sx2 = b_hat
    x2=Solucion_SEL_QR_nxn(S,b_h)

    # 4. Resolver A11x1 = b1-A12x2
    x1=Solucion_SEL_QR_nxn(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

#eliminacion_bloques.__doc__ =busca_ayuda("eliminacion_bloques")

### Pruebas

In [3]:
# Leer el conjunto de datos Iris
from sklearn import datasets
iris = datasets.load_iris()
# dado que esta guardado como un objeto tipo lista, conservar sólo los datos
datos_iris = iris['data']
datos_iris

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [4]:
mean_vec = np.mean(datos_iris, axis=0)

### Sacar SVD solo para comparar. Calcular sobre la matriz de datos originales centrados (sobre esta matriz se debe hacer svd)


In [5]:
u, s, vh = np.linalg.svd(datos_iris - mean_vec, full_matrices=False)

In [6]:
#print(u)
print("------------------------------------------------")
print(s)
print("------------------------------------------------")
print(vh)

------------------------------------------------
[25.09996044  6.01314738  3.41368064  1.88452351]
------------------------------------------------
[[ 0.36138659 -0.08452251  0.85667061  0.3582892 ]
 [-0.65658877 -0.73016143  0.17337266  0.07548102]
 [ 0.58202985 -0.59791083 -0.07623608 -0.54583143]
 [ 0.31548719 -0.3197231  -0.47983899  0.75365743]]


## Funcion creada para este proyecto final

Función que ocupa algoritm QR para calcular los eigenvectores de una matriz

In [7]:
def eigenvectores_eigenvalores_QR(data,niter,tolerancia = 10**-8):
    """
    Función para obtener los eigenvectores y eigenvalores de la matriz de covarianzas
    params: A			matriz de datos
            niter: número de iteraciones máximas 
    
    
    return:     eigenvalores 
                eigenvalores 
                
    Depende de las funciones ue calculan la factorización QR
    """

    # Notar que crear la matriz de covarianzas por variables, se hace transpuesta(X) %*% X 
    # La matriz inicial será la matriz de covarianzas
    n = data.shape[0]
    A= np.array(data)
    
    mean_vec = np.mean(A, axis=0)
    #A_centered = A - A.mean(axis=0)
    
    #cov_mat = (A_centered - mean_vec).T.dot((A_centered - mean_vec)) / (n-1)
    cov_mat = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    #C = ((np.array(A).T)@(np.array(A))) / (n-1)
    #A_centered = A - A.mean(axis=0)
    
    matriz_cov = cov_mat   
    A0 = cov_mat.copy()
    
    # La matriz de covarianza ya es simetrica, no necesitamos rellenar columnas de ceros
    # Completamos la matriz A con columnas de ceros para que sea cuadrada
    #n, p = A.shape
    #columnas = n - p
    #ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    #A0 = np.append(A, ceros, axis = 1)
    # Factorización inicial
    Qk,Rk =matriz_Q_R(A0)
    
    # Hacer una copia de la matriz Q inicial
    Q = Qk.copy()
    
    valor = 10
    
    #iniciamos ciclo
    for i in range(niter):
        
        #Definimos la nueva Ak+1
        Ak = Rk@Qk
        #calculamos la factorización QR sobre la nueva Ak
        Qk,Rk =matriz_Q_R(Ak)
        
        # Se usan los valores distintos de la diagonal para corroborar la tolerancia deseada
        valor = np.sum(np.abs(Qk)) - np.trace(np.abs(Qk))
        
        
        # Detener si se logra la tolerancia deseada
        # Notar que en esta primera prueba no se está actualizando  el valor que checa la tolerancia
        if (valor<tolerancia):
            print("se alcanzo la torelacia deseada")
            break
    
    #se acaba el for
    
    #La diagonal de Ak nos regresa los eigenvalores
    eigenvalores = np.diagonal(Ak)
        
    return eigenvalores, Q

In [8]:
# De momento la función regresa los eigenvalores y la última matriz Q, esta matriz podría no pedirse, ahorita está de más
# sólo era para revisar que valores tiene
E1, Q1 = eigenvectores_eigenvalores_QR(datos_iris,120)

se alcanzo la torelacia deseada


In [9]:
print(E1)
print("----------------------------------------------")
print(Q1)


[4.22824171 0.24267075 0.0782095  0.02383509]
----------------------------------------------
[[ 0.44612248  0.5011469  -0.72000481  0.17725574]
 [-0.0276082   0.83426186  0.51665264 -0.1905649 ]
 [ 0.82908872 -0.22692966  0.24542048 -0.44820041]
 [ 0.33589345 -0.0369034   0.39297826  0.85520864]]


In [10]:
E1/sum(E1)

array([0.92461872, 0.05306648, 0.01710261, 0.00521218])

Modificar la función para calcular de forma correcta Q

In [17]:
def eigenvectores_eigenvalores_QR2(data,niter,tolerancia = 10**-8):
    """
    Función para obtener los eigenvectores y eigenvalores de la matriz de covarianzas
    params: A			matriz de datos
            niter: número de iteraciones máximas 
    
    
    return:     eigenvalores 
                eigenvalores 
                
    Depende de las funciones ue calculan la factorización QR
    """

    # Notar que crear la matriz de covarianzas por variables, se hace transpuesta(X) %*% X 
    # La matriz inicial será la matriz de covarianzas
    n = data.shape[0]
    A= np.array(data)
    
    mean_vec = np.mean(A, axis=0)
    #A_centered = A - A.mean(axis=0)
    
    #cov_mat = (A_centered - mean_vec).T.dot((A_centered - mean_vec)) / (n-1)
    C = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    #C = ((np.array(A).T)@(np.array(A))) / (n-1)
    #A_centered = A - A.mean(axis=0)
      
    A0 = C.copy()
    
    # La matriz de covarianza ya es simetrica, no necesitamos rellenar columnas de ceros
    # Completamos la matriz A con columnas de ceros para que sea cuadrada
    #n, p = A.shape
    #columnas = n - p
    #ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    #A0 = np.append(A, ceros, axis = 1)
    # Factorización inicial
    Qk,Rk =matriz_Q_R(A0)
    
    # Hacer una copia de la matriz Q inicial
    Q = Qk.copy()
    
    valor = 10
    
    #iniciamos ciclo
    for i in range(niter):
        
        #Definimos la nueva Ak+1
        Ak = Rk@Qk
        #calculamos la factorización QR sobre la nueva Ak
        Qk,Rk =matriz_Q_R(Ak)
        
        # Actualizar la matriz Q
        Q = Q@Qk
        
        
        # Se usan los valores distintos de la diagonal para corroborar la tolerancia deseada
        valor = np.sum(np.abs(Qk)) - np.trace(np.abs(Qk))
        
        
        # Detener si se logra la tolerancia deseada
        # Notar que en esta primera prueba no se está actualizando  el valor que checa la tolerancia
        if (valor<tolerancia):
            print("se alcanzo la torelacia deseada")
            break
    
    #se acaba el for
    
    #La diagonal de Ak nos regresa los eigenvalores
    eigenvalores = np.diagonal(Ak)
        
    return eigenvalores, Q

In [18]:
E2, Q2 = eigenvectores_eigenvalores_QR2(datos_iris,120)

se alcanzo la torelacia deseada


In [19]:
print(E2)
print("----------------------------------------------")
print(Q2)


[4.22824171 0.24267075 0.0782095  0.02383509]
----------------------------------------------
[[ 0.36138659  0.65658877 -0.58202985  0.31548719]
 [-0.08452251  0.73016144  0.59791083 -0.3197231 ]
 [ 0.85667061 -0.17337266  0.07623608 -0.47983899]
 [ 0.3582892  -0.07548102  0.54583143  0.75365743]]


In [20]:
E2/sum(E2)

array([0.92461872, 0.05306648, 0.01710261, 0.00521218])

In [21]:
E2**2

array([1.78780279e+01, 5.88890919e-02, 6.11672590e-03, 5.68111657e-04])

Usar numpy para cacular los valores y eigen vectores de una matriz de covarianzas

In [22]:
mean_vec = np.mean(datos_iris, axis=0)

# Matriz de covarianzas
C = (datos_iris - mean_vec).T.dot((datos_iris - mean_vec)) / (datos_iris.shape[0]-1)
C

array([[ 0.68569351, -0.042434  ,  1.27431544,  0.51627069],
       [-0.042434  ,  0.18997942, -0.32965638, -0.12163937],
       [ 1.27431544, -0.32965638,  3.11627785,  1.2956094 ],
       [ 0.51627069, -0.12163937,  1.2956094 ,  0.58100626]])

In [23]:
eigenvalores, eigenvectotes= np.linalg.eig(C)

In [24]:
print(eigenvalores)
print("----------------------------------------------")
print(eigenvectotes)

[4.22824171 0.24267075 0.0782095  0.02383509]
----------------------------------------------
[[ 0.36138659 -0.65658877 -0.58202985  0.31548719]
 [-0.08452251 -0.73016143  0.59791083 -0.3197231 ]
 [ 0.85667061  0.17337266  0.07623608 -0.47983899]
 [ 0.3582892   0.07548102  0.54583143  0.75365743]]


In [111]:
def PCA_from_QR(data,niter):
    """
    Función para PCA a partir de los eigenvectores  
    params: A:			matriz de datos
            niter:      número de iteraciones máximas    
    
    
    return:     componentes		Los coeficientes para calcular los componentes principales (eigenvectores de la matriz de covarianzas)
                Z			Los datos transformados (componentes principales)
                varianza_explicada	La varianza explicada por cada componente principal
    
    Depende de la función: eigenvectores_QR
    """
    

    
    # Calcular algoritmo QR
    E, Q = eigenvectores_eigenvalores_QR2(data,niter)
    
    
    # Los componentes (coeficientes)
    componentes = Q.T
    
    # Los datos transformados (componentes principales)
    Z = data@Q
    
    # La varianza explicada
    varianza_explicada = E/np.sum(E)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = data.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1
    
    # regresar 4 objetos
    #return E[:num_componentes], componentes[:num_componentes].T, Z[:,:num_componentes], varianza_explicada[:num_componentes]
    return E, componentes, Z, varianza_explicada #, varianza_acumulada, num_componentes

### Probar Función

In [112]:
eigenvalores, coeficientes, Z, varianza_explicada = PCA_from_QR(datos_iris,120)

se alcanzo la torelacia deseada


In [113]:
eigenvalores

array([4.22824171, 0.24267075, 0.0782095 , 0.02383509])

In [39]:
coeficientes

array([[ 0.36138659],
       [-0.08452251],
       [ 0.85667061],
       [ 0.3582892 ]])

In [40]:
Z

array([[2.81823951],
       [2.78822345],
       [2.61337456],
       [2.75702228],
       [2.7736486 ],
       [3.2215055 ],
       [2.68182738],
       [2.87622016],
       [2.6159824 ],
       [2.82960933],
       [2.99541804],
       [2.8896099 ],
       [2.71625587],
       [2.27856139],
       [2.85761474],
       [3.1163261 ],
       [2.87883726],
       [2.85406843],
       [3.30254481],
       [2.91437873],
       [3.19210892],
       [2.9586599 ],
       [2.28642572],
       [3.19963195],
       [3.14661108],
       [2.99569623],
       [3.03354506],
       [2.94004523],
       [2.86283042],
       [2.87037575],
       [2.91496666],
       [3.09243264],
       [2.8535028 ],
       [2.90362838],
       [2.86543825],
       [2.63612348],
       [2.87712708],
       [2.70168102],
       [2.52186309],
       [2.91235882],
       [2.73226271],
       [2.65299643],
       [2.50495859],
       [3.09675065],
       [3.29287589],
       [2.78791371],
       [2.96421687],
       [2.662

In [41]:
varianza_explicada

array([0.92461872])

Calcular los componentes principales a partir de los 'loadings' obtenidos

In [48]:
mean_vec = np.mean(datos_iris, axis=0)

# datos centrados
datos_centrados = (datos_iris - mean_vec)

In [49]:
print(datos_iris[:5])
print('--------------------------------------------------------------------------')
print(mean_vec)
print('--------------------------------------------------------------------------')
print(datos_centrados[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
--------------------------------------------------------------------------
[5.84333333 3.05733333 3.758      1.19933333]
--------------------------------------------------------------------------
[[-0.74333333  0.44266667 -2.358      -0.99933333]
 [-0.94333333 -0.05733333 -2.358      -0.99933333]
 [-1.14333333  0.14266667 -2.458      -0.99933333]
 [-1.24333333  0.04266667 -2.258      -0.99933333]
 [-0.84333333  0.54266667 -2.358      -0.99933333]]


In [51]:
coeficientes

array([[ 0.36138659],
       [-0.08452251],
       [ 0.85667061],
       [ 0.3582892 ]])

In [52]:
datos_centrados@coeficientes

array([[-2.68412563],
       [-2.71414169],
       [-2.88899057],
       [-2.74534286],
       [-2.72871654],
       [-2.28085963],
       [-2.82053775],
       [-2.62614497],
       [-2.88638273],
       [-2.6727558 ],
       [-2.50694709],
       [-2.61275523],
       [-2.78610927],
       [-3.22380374],
       [-2.64475039],
       [-2.38603903],
       [-2.62352788],
       [-2.64829671],
       [-2.19982032],
       [-2.5879864 ],
       [-2.31025622],
       [-2.54370523],
       [-3.21593942],
       [-2.30273318],
       [-2.35575405],
       [-2.50666891],
       [-2.46882007],
       [-2.56231991],
       [-2.63953472],
       [-2.63198939],
       [-2.58739848],
       [-2.4099325 ],
       [-2.64886233],
       [-2.59873675],
       [-2.63692688],
       [-2.86624165],
       [-2.62523805],
       [-2.80068412],
       [-2.98050204],
       [-2.59000631],
       [-2.77010243],
       [-2.84936871],
       [-2.99740655],
       [-2.40561449],
       [-2.20948924],
       [-2

### Siguiente prueba. Usar un subconjunto de los datos limpios

In [53]:
# importar datos
import pandas as pd
df = pd.read_csv('/home/leonardo/Documentos/ITAM/MNO/Final-Project-MNO-2020/data/datos_limpios.csv', nrows = 20000)
#df = pd.read_csv('/home/leonardo/Documentos/ITAM/MNO/Final-Project-MNO-2020/data/datos_limpios.csv')

In [54]:
# Conservar variables de la sección 3
seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']]
seccion_3.head()

,p10,p11,p12,p13,p14
0,2.0,1.0,1.0,2.0,2.0
1,4.0,3.0,3.0,5.0,6.0
2,4.0,1.0,2.0,5.0,5.0
3,2.0,1.0,1.0,3.0,3.0
4,2.0,2.0,3.0,3.0,4.0


Probar la funcion que calcula eigenvalores 

In [55]:
E1, Q1 = eigenvectores_eigenvalores_QR2(seccion_3,120)

se alcanzo la torelacia deseada


Probar función que hace PCA

In [59]:
eigenvalores, coeficientes, Z, varianza_explicada, var_acum, n = PCA_from_QR(seccion_3,120)

se alcanzo la torelacia deseada


In [60]:
eigenvalores

array([4.70597231, 1.53945229, 0.87015472, 0.52849951, 0.39522596])

In [61]:
coeficientes

array([[ 0.06558817,  0.28749878,  0.4607722 ,  0.58625881,  0.59752175],
       [-0.21036303,  0.49739437,  0.66144939, -0.3307176 , -0.40181689],
       [-0.9562346 , -0.02828963, -0.15949514,  0.24365987,  0.00250049],
       [ 0.13371182,  0.7579274 , -0.53349033,  0.26588056, -0.22882919],
       [ 0.13849954, -0.30771585,  0.2003074 ,  0.64564997, -0.65508923]])

In [62]:
Z

,0,1,2,3,4
0,3.247008,-0.726951,-1.607933,0.565963,0.150712
1,9.023590,-1.429410,-3.154990,1.164586,-0.470513
2,7.390299,-2.683831,-2.941417,0.411051,0.599701
3,4.430789,-1.459486,-1.361773,0.603015,0.141273
4,6.237354,-0.041010,-1.706552,0.065132,-0.420917
...,...,...,...,...,...
19995,10.728041,-2.282299,-1.708936,1.333806,0.027198
19996,10.380644,-1.997890,-3.040036,-0.089780,0.028071
19997,6.505280,-1.525002,-3.457026,0.637217,-0.999315
19998,10.358120,-0.601284,-3.099115,1.654904,0.067729


In [63]:
varianza_explicada

array([0.58537056, 0.19149072, 0.10823756, 0.06573945, 0.04916171])

In [64]:
var_acum

array([0.58537056, 0.77686128, 0.88509884, 0.95083829, 1.        ])

In [65]:
n

2

In [81]:
def PCA_from_QR2(data,niter):
    """
    Función para PCA a partir de los eigenvectores  
    params: A:			matriz de datos
            niter:      número de iteraciones máximas    
    
    
    return:     componentes		Los coeficientes para calcular los componentes principales (eigenvectores de la matriz de covarianzas)
                Z			Los datos transformados (componentes principales)
                varianza_explicada	La varianza explicada por cada componente principal
    
    Depende de la función: eigenvectores_QR
    """
    

    
    # Calcular algoritmo QR
    E, Q = eigenvectores_eigenvalores_QR2(data,niter)
    
    
    # Los componentes (coeficientes)
    componentes = Q.T
    
    # Los datos transformados (componentes principales)
    # Aquí marcaba error al filtrar porque no se reconocia a Z como numpy array
    Z = np.array(data)@Q
    
    # La varianza explicada
    varianza_explicada = E/np.sum(E)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = data.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1
    
    # regresar 4 objetos
    return E[:num_componentes], componentes[:num_componentes].T, Z[:,:num_componentes], varianza_explicada[:num_componentes], varianza_acumulada, num_componentes
    #return E, componentes, Z, varianza_explicada, varianza_acumulada, num_componentes

In [82]:
eigenvalores, coeficientes, Z, varianza_explicada, var_acum, n = PCA_from_QR2(seccion_3,120)

se alcanzo la torelacia deseada


In [83]:
eigenvalores

array([4.70597231, 1.53945229])

In [84]:
coeficientes

array([[ 0.06558817, -0.21036303],
       [ 0.28749878,  0.49739437],
       [ 0.4607722 ,  0.66144939],
       [ 0.58625881, -0.3307176 ],
       [ 0.59752175, -0.40181689]])

In [85]:
Z

array([[ 3.24700842, -0.72695129],
       [ 9.02359012, -1.4294102 ],
       [ 7.39029862, -2.68383144],
       ...,
       [ 6.50527979, -1.52500186],
       [10.35811991, -0.60128405],
       [ 9.60984893, -1.7601278 ]])

In [86]:
varianza_explicada

array([0.58537056, 0.19149072])

In [87]:
var_acum

array([0.58537056, 0.77686128, 0.88509884, 0.95083829, 1.        ])

In [88]:
n

2

## Afinar funciones

No sacar la matriz de covarianzas en esta función

In [93]:
def eigenvectores_eigenvalores_QR_vf(data,niter,tolerancia = 10**-8):
    """
    Función para obtener los eigenvectores y eigenvalores de una matriz cualquiera
    params: data			matriz de datos
            niter: número de iteraciones máximas 
    
    
    return:     eigenvalores 
                eigenvalores 
                
    Depende de las funciones ue calculan la factorización QR
    """

    # convertir a array
    A = np.array(data)
    
    # La matriz de covarianza ya es simetrica, no necesitamos rellenar columnas de ceros
    # Completamos la matriz A con columnas de ceros para que sea cuadrada
    n, p = A.shape
    columnas = n - p
    ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    A0 = np.append(A, ceros, axis = 1)
    # Factorización inicial
    Qk,Rk =matriz_Q_R(A0)
    
    # Hacer una copia de la matriz Q inicial
    Q = Qk.copy()
    
    valor = 10
    
    #iniciamos ciclo
    for i in range(niter):
        
        #Definimos la nueva Ak+1
        Ak = Rk@Qk
        #calculamos la factorización QR sobre la nueva Ak
        Qk,Rk =matriz_Q_R(Ak)
        
        # Actualizar la matriz Q
        Q = Q@Qk
        
        
        # Se usan los valores distintos de la diagonal para corroborar la tolerancia deseada
        valor = np.sum(np.abs(Qk)) - np.trace(np.abs(Qk))
        
        
        # Detener si se logra la tolerancia deseada
        # Notar que en esta primera prueba no se está actualizando  el valor que checa la tolerancia
        if (valor<tolerancia):
            print("se alcanzo la torelacia deseada")
            break
    
    #se acaba el for
    
    #La diagonal de Ak nos regresa los eigenvalores
    eigenvalores = np.diagonal(Ak)
        
    return eigenvalores, Q

Probar con una matriz no simetrica

In [114]:
e, q = eigenvectores_eigenvalores_QR_vf(datos_iris,10)

In [115]:
print(e[:5])
print('--------------------------------------------------------------------------')
print(q[:2])

[ 9.83727522e+00 -2.50616177e-01  5.22065754e-02 -3.88656187e-02
 -1.28154255e-28]
--------------------------------------------------------------------------
[[ 5.91447560e-02  1.22776067e-01 -4.31753192e-02  1.29168997e-02
  -1.65021301e-01  4.31296508e-02 -2.11293277e-01 -1.90279866e-01
   2.30726729e-01  2.06292404e-01 -2.09659665e-01 -6.96975994e-02
  -4.15914938e-02  1.46616734e-01  6.38124417e-02  3.86459959e-02
  -3.54699982e-02  1.11414428e-01 -1.94934436e-01  1.60724899e-01
   5.50200399e-02 -1.58450988e-01  4.51076355e-02 -2.28494072e-01
   2.24913033e-02  5.41102022e-02  1.02028696e-01 -7.05628950e-03
   9.71789702e-02  3.71431779e-02  1.36110111e-01 -8.43499847e-03
  -5.06709442e-02  6.74326952e-02  9.10873752e-02  1.35912490e-01
  -6.05438049e-02 -1.20039862e-02  2.33805015e-02 -4.77701298e-03
   2.68727407e-02 -8.50587684e-02 -2.96412948e-05 -5.43792365e-02
  -2.23571751e-02  2.30803156e-02  6.08937207e-02 -1.33549312e-01
   5.34621697e-02  1.08224195e-02 -8.51911385e-02 

probar con una matriz simétrica

In [116]:
mean_vec = np.mean(datos_iris, axis=0)

# Matriz de covarianzas
C = (datos_iris - mean_vec).T.dot((datos_iris - mean_vec)) / (datos_iris.shape[0]-1)
C

array([[ 0.68569351, -0.042434  ,  1.27431544,  0.51627069],
       [-0.042434  ,  0.18997942, -0.32965638, -0.12163937],
       [ 1.27431544, -0.32965638,  3.11627785,  1.2956094 ],
       [ 0.51627069, -0.12163937,  1.2956094 ,  0.58100626]])

In [117]:
e, q = eigenvectores_eigenvalores_QR_vf(C,10)

In [118]:
print(e)
print('--------------------------------------------------------------------------')
print(q)

[4.22824171 0.24267075 0.0782095  0.02383509]
--------------------------------------------------------------------------
[[ 0.36138659  0.65658759 -0.58203167  0.31548631]
 [-0.08452251  0.73016265  0.59790983 -0.31972219]
 [ 0.85667061 -0.17337251  0.07623716 -0.47983887]
 [ 0.3582892  -0.07547991  0.54583044  0.75365826]]


In [144]:
def PCA_from_QR_vf(data,niter):
    """
    Función para PCA a partir de los eigenvectores  
    params: data:			matriz de datos
            niter:      número de iteraciones máximas    
    
    
    return:     componentes		Los coeficientes para calcular los componentes principales (eigenvectores de la matriz de covarianzas)
                Z			Los datos transformados (componentes principales)
                varianza_explicada	La varianza explicada por cada componente principal
    
    Depende de la función: eigenvectores_QR
    """
    
    # convertir a array
    A = np.array(data)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)

    # Matriz de Covarianzas
    #C = (datos_centrados.T@datos_centrados)/(datos_centrados.shape[0]-1)
    C = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    
    # Calcular algoritmo QR
    E, Q = eigenvectores_eigenvalores_QR_vf(C,niter)
    
    
    # Los componentes (coeficientes)
    componentes = Q.T
    
    # Los datos transformados (componentes principales)
    # Aquí marcaba error al filtrar porque no se reconocia a Z como numpy array
    Z = datos_centrados@Q
    
    # La varianza explicada
    varianza_explicada = E/np.sum(E)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = data.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1
    
    # regresar 4 objetos
    return E[:num_componentes], componentes[:num_componentes].T, Z[:,:num_componentes], varianza_explicada[:num_componentes] #, varianza_acumulada, num_componentes
    #return E, componentes, Z, varianza_explicada, varianza_acumulada, num_componentes

Probar función

In [145]:
eigenvalores, coeficientes, Z, varianza_explicada = PCA_from_QR_vf(datos_iris,120)

se alcanzo la torelacia deseada


In [146]:
eigenvalores

array([4.22824171])

In [147]:
coeficientes

array([[ 0.36138659],
       [-0.08452251],
       [ 0.85667061],
       [ 0.3582892 ]])

In [148]:
Z

array([[-2.68412563],
       [-2.71414169],
       [-2.88899057],
       [-2.74534286],
       [-2.72871654],
       [-2.28085963],
       [-2.82053775],
       [-2.62614497],
       [-2.88638273],
       [-2.6727558 ],
       [-2.50694709],
       [-2.61275523],
       [-2.78610927],
       [-3.22380374],
       [-2.64475039],
       [-2.38603903],
       [-2.62352788],
       [-2.64829671],
       [-2.19982032],
       [-2.5879864 ],
       [-2.31025622],
       [-2.54370523],
       [-3.21593942],
       [-2.30273318],
       [-2.35575405],
       [-2.50666891],
       [-2.46882007],
       [-2.56231991],
       [-2.63953472],
       [-2.63198939],
       [-2.58739848],
       [-2.4099325 ],
       [-2.64886233],
       [-2.59873675],
       [-2.63692688],
       [-2.86624165],
       [-2.62523805],
       [-2.80068412],
       [-2.98050204],
       [-2.59000631],
       [-2.77010243],
       [-2.84936871],
       [-2.99740655],
       [-2.40561449],
       [-2.20948924],
       [-2

In [149]:
varianza_explicada

array([0.92461872])